In [40]:
import tensorflow as tf

# TODO(zongheng): read from argparse.
# TODO(zongheng): visualization via tensorboard.
# TODO(zongheng): run eval on test set

# Toy dataset.
TRAIN_PATH = "data/train-5rel-10numAttrs-26totalAttrs-100maxTblSize-100buckets-30.csv"
TEST_PATH = "data/test-5rel-10numAttrs-26totalAttrs-100maxTblSize-100buckets-30.csv"
NUM_FIELDS = 38
net = [64]
lr = 5e-4
batch_size = 1
num_steps = 7000

# 1000 examples in training.
# Min train loss < 0.6
TRAIN_PATH = "data/train-5rel-10numAttrs-17totalAttrs-100maxTblSize-100buckets-3000.csv"
TEST_PATH = "data/test-5rel-10numAttrs-17totalAttrs-100maxTblSize-100buckets-3000.csv"
NUM_FIELDS = 29
net = [64]
lr = 3e-3
batch_size = 128
num_steps = 5500

In [41]:
dataset = tf.data.TextLineDataset(TRAIN_PATH)

def ParseLine(line):
    fields = tf.decode_csv(line, [[0.0]] * NUM_FIELDS)
    # Last number is label.
    return tf.reshape(tf.stack(fields[:-1]), [-1]), fields[-1]

dataset = dataset.map(ParseLine)
# dataset = dataset.shuffle(2000).repeat().batch(batch_size)
dataset = dataset.shuffle(20000).batch(batch_size).repeat()

print(dataset.output_types)
print(dataset.output_shapes)

iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
iterator_init = iterator.make_initializer(dataset)
feature_batch, label_batch = iterator.get_next()

(tf.float32, tf.float32)
(TensorShape([Dimension(None), Dimension(28)]), TensorShape([Dimension(None)]))


In [42]:
feature_batch, label_batch

(<tf.Tensor 'IteratorGetNext_6:0' shape=(?, 28) dtype=float32>,
 <tf.Tensor 'IteratorGetNext_6:1' shape=(?,) dtype=float32>)

In [43]:
# Network.
for i in range(len(net)):
    dense = tf.layers.dense(
        dense if i > 0 else feature_batch,
        net[i],
        kernel_initializer=tf.truncated_normal_initializer,
        activation=tf.nn.relu)
out = tf.layers.dense(
    dense,
    1,
    kernel_initializer=tf.truncated_normal_initializer)

loss = tf.reduce_mean(tf.square(out - label_batch))
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_step = optimizer.minimize(
    loss, global_step=tf.train.get_global_step())

In [44]:
sess = tf.InteractiveSession()

In [45]:
# Train.

sess.run(tf.global_variables_initializer())
sess.run(iterator_init)

for i in range(num_steps):
    _, loss_val = sess.run([train_step, loss])
    if i % 100 == 0:
        print('step', i, 'loss', loss_val)
    if loss_val < 1e-5:
        print('Done:', 'step', i, 'loss', loss_val)
        break

step 0 loss 6318.671
step 100 loss 16.696434
step 200 loss 5.845072
step 300 loss 2.6955657
step 400 loss 2.4313056
step 500 loss 1.586024
step 600 loss 1.6442583
step 700 loss 1.4798006
step 800 loss 1.5943727
step 900 loss 1.0702156
step 1000 loss 0.78816366
step 1100 loss 0.8568829
step 1200 loss 0.98739046
step 1300 loss 0.8032566
step 1400 loss 0.9122672
step 1500 loss 0.9396315
step 1600 loss 0.78366625
step 1700 loss 0.6793132
step 1800 loss 1.1123011
step 1900 loss 0.6506945
step 2000 loss 0.97821295
step 2100 loss 0.77451754
step 2200 loss 0.8072728
step 2300 loss 0.8339112
step 2400 loss 1.0729866
step 2500 loss 1.1488535
step 2600 loss 0.8714757
step 2700 loss 0.87070477
step 2800 loss 1.4514767
step 2900 loss 0.6043769
step 3000 loss 0.6341361
step 3100 loss 1.5737374
step 3200 loss 0.6084759
step 3300 loss 0.8943867
step 3400 loss 0.9178779
step 3500 loss 0.6253151
step 3600 loss 0.9353164
step 3700 loss 0.64219785
step 3800 loss 0.7953
step 3900 loss 0.7631886
step 4000 l